In [1]:
from datetime import datetime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import metpy.calc as mpcalc
from metpy.units import units
import numpy as np
from scipy.ndimage import gaussian_filter
import swiftascmaps as swift
import xarray as xr

In [2]:
# Open NCEP GFS dataset; change name accordingly to yours
ds = xr.open_dataset('US_MESOSCALE/081416_0600H.f00_Analysis.nc4')
ds

<xarray.Dataset>
Dimensions:                                                          (sigma: 1,
                                                                      potential_vorticity_surface: 2,
                                                                      pressure_difference_layer2: 2,
                                                                      bounds_dim: 2,
                                                                      latitude: 29,
                                                                      height_above_ground1: 3,
                                                                      ...
                                                                      height_above_ground_layer: 1,
                                                                      isobaric3: 31,
                                                                      isobaric1: 21,
                                                                      time: 1,
                                                                      height_above_ground_layer1: 1,
                                                                      longitude: 33)
Coordinates: (12/13)
  * sigma                                                            (sigma) float64 ...
  * potential_vorticity_surface                                      (potential_vorticity_surface) float64 ...
  * pressure_difference_layer2                                       (pressure_difference_layer2) float64 ...
  * latitude                                                         (latitude) float32 ...
  * height_above_ground1                                             (height_above_ground1) float64 ...
  * height_above_ground3                                             (height_above_ground3) float64 ...
    ...                                                               ...
    reftime                                                          datetime64[ns] ...
  * isobaric3                                                        (isobaric3) float64 ...
  * isobaric1                                                        (isobaric1) float64 ...
  * time                                                             (time) datetime64[ns] ...
  * height_above_ground_layer1                                       (height_above_ground_layer1) float64 ...
  * longitude                                                        (longitude) float32 ...
Dimensions without coordinates: bounds_dim
Data variables: (12/29)
    pressure_difference_layer2_bounds                                (pressure_difference_layer2, bounds_dim) float64 ...
    height_above_ground_layer_bounds                                 (height_above_ground_layer, bounds_dim) float64 ...
    height_above_ground_layer1_bounds                                (height_above_ground_layer1, bounds_dim) float64 ...
    U-Component_Storm_Motion_height_above_ground_layer               (time, height_above_ground_layer1, latitude, longitude) float32 ...
    Vertical_velocity_pressure_isobaric                              (time, isobaric1, latitude, longitude) float32 ...
    Convective_inhibition_surface                                    (time, latitude, longitude) float32 ...
    ...                                                               ...
    u-component_of_wind_isobaric                                     (time, isobaric3, latitude, longitude) float32 ...
    Dewpoint_temperature_height_above_ground                         (time, height_above_ground3, latitude, longitude) float32 ...
    Storm_relative_helicity_height_above_ground_layer                (time, height_above_ground_layer, latitude, longitude) float32 ...
    V-Component_Storm_Motion_height_above_ground_layer               (time, height_above_ground_layer1, latitude, longitude) float32 ...
    Vertical_velocity_pressure_sigma                                 (time, sigma, latitude, longitude) float32 ...
    LatLon_721X1440-0p13S-180p00E     

In [3]:
# Grab lat/lon values (NAM will be 2D)
lats = ds['latitude']
lons = ds['longitude']

# Select and grab data
mslp = ds['MSLP_Eta_model_reduction_msl']
ustorm = ds['U-Component_Storm_Motion_height_above_ground_layer']
vstorm = ds['V-Component_Storm_Motion_height_above_ground_layer']
srh = ds['Storm_relative_helicity_height_above_ground_layer']

# Select and grab MSLP. Apply gaussian_filter.
mslp = gaussian_filter(mslp.data[0], sigma=3.0) * units.pascal
mslph = mslp.to('hPa')

# Select wind components of storm motion 
ustorm_3k = gaussian_filter(ustorm.sel(height_above_ground_layer1=3000).data[0], sigma=3.0) * units('m/s')
vstorm_3k = gaussian_filter(vstorm.sel(height_above_ground_layer1=3000).data[0], sigma=3.0) * units('m/s')

# Add Storm Relative Helicity
srh_sm = gaussian_filter(srh.sel(height_above_ground_layer=1500).data[0], sigma=3.0) * units('J/kg')

# Create a clean datetime object for plotting based on time of Geopotential heights
vtime = datetime.strptime(str(ds.time.data[0].astype('datetime64[ms]')),
                          '%Y-%m-%dT%H:%M:%S.%f')

In [ ]:
# Set up the projection that will be used for plotting
mapcrs = ccrs.PlateCarree()

# Set up the projection of the data; if lat/lon then PlateCarree is what you want
datacrs = ccrs.PlateCarree()

# Start the figure and create plot axes with proper projection
fig = plt.figure(1, figsize=(14, 12))
ax = plt.subplot(111, projection=datacrs)
ax.set_extent([117, 125, 13, 20], ccrs.PlateCarree())

# Add geopolitical boundaries for map reference
states_provinces = cfeature.NaturalEarthFeature(category='cultural', name='admin_1_states_provinces_lines', 
                                                scale='10m', facecolor='none', linewidth=0.75)
ax.add_feature(states_provinces, edgecolor='k')
ax.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=1.15)
ax.add_feature(cfeature.STATES.with_scale('50m'))
ax.add_feature(cfeature.LAND)

# Add Storm Relative Helicity contours
srh_range = np.arange(0, 140, 5)
srh = ax.contourf(lons, lats, srh_sm, srh_range, cmap='swift.midnights_r', transform=datacrs)
cbar = plt.colorbar(srh, orientation='vertical', pad=0.01, aspect=35, shrink=0.664)
cbar.ax.minorticks_on()
cbar.ax.tick_params(which='major', length=10, width=1.5, top=True, right=True, direction='in')
cbar.ax.tick_params(which='minor', length=5, width=1, top=True, right=True, direction='in')

# Add Storm Motion using U and V-components
ax.barbs(lons, lats, ustorm_3k.to('kt').m, vstorm_3k.to('kt').m, pivot='middle',
         color='black', regrid_shape=17, transform=datacrs)

# Plot MSLP Contour
MSLP_range = np.arange(998, 1005, 0.5)
prs = ax.contour(lons, lats, mslph, MSLP_range, colors='k',
                 transform=datacrs, zorder=1)
ax.clabel(prs, fmt='%d')

# Create a/the line(s) that highlights the surface trough
outline=pe.withStroke(linewidth=5, foreground='k')

point1 = dict(lon=120.40, lat=17.80)
point2 = dict(lon=120.00, lat=17.10)
lon_p = [point1['lon'], point2['lon']]
lat_p = [point1['lat'], point2['lat']]
ax.plot(lon_p, lat_p, color='red', lw=3, ls='--', 
        path_effects=[outline, pe.Normal()], transform=ccrs.Geodetic())

point3 = dict(lon=120.00, lat=17.10)
point4 = dict(lon=120.00, lat=15.60)
lon_p1 = [point3['lon'], point4['lon']]
lat_p1 = [point3['lat'], point4['lat']]
ax.plot(lon_p1, lat_p1, color='red', lw=3, ls='--', 
        path_effects=[outline, pe.Normal()], transform=ccrs.Geodetic())

point5 = dict(lon=120.00, lat=15.60)
point6 = dict(lon=120.45, lat=13.60)
lon_p2 = [point5['lon'], point6['lon']]
lat_p2 = [point5['lat'], point6['lat']]
ax.plot(lon_p2, lat_p2, color='red', lw=3, ls='--', 
        path_effects=[outline, pe.Normal()], transform=ccrs.Geodetic())

point7 = dict(lon=120.45, lat=13.60)
point8 = dict(lon=121.10, lat=11.70)
lon_p3 = [point7['lon'], point8['lon']]
lat_p3 = [point7['lat'], point8['lat']]
ax.plot(lon_p3, lat_p3, color='red', lw=3, ls='--', 
        path_effects=[outline, pe.Normal()], transform=ccrs.Geodetic())

# Make some nice titles for the plot (one right, one left)
ax.set_title('NCEP FNL: MSLP (hPa), V$_{Storm Motion}$ (kt), and SRH (J/kg)', loc='left')
ax.set_title('Time: {} UTC'.format(vtime), loc='right')

# Add marker of Manila, NCR via its lats and lons
manila_lon, manila_lat = 120.9842, 14.5995
ax.plot(manila_lon, manila_lat, color='blue', markersize=7, 
        markeredgewidth=1.5, markeredgecolor='k', marker='o', transform=datacrs)

# Set X and Y-ticks for Latitude and Longitude Coordinates
ax.set_yticks(lats, 5, crs=ccrs.PlateCarree())
ax.set_xticks(lons, 5, crs=ccrs.PlateCarree())
lat_formatter = cticker.LatitudeFormatter()
long_formatter = cticker.LongitudeFormatter()
ax.yaxis.set_major_formatter(lat_formatter)
ax.xaxis.set_major_formatter(long_formatter)
ax.set_xticks(ax.get_xticks()[1:8]) #Hide end points of longitude tickmarks
ax.set_yticks(ax.get_yticks()[1:7]) #Hide end points of latitude tickmarks
ax.minorticks_on() 
ax.tick_params(which='major', length=10, width=1.5, top=True, right=True, direction='in')
ax.tick_params(which='minor', length=5, width=1, top=True, right=True, direction='in')

# Adjust image and show
plt.subplots_adjust(bottom=0, top=1)
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['figure.dpi'] = 500
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.savefig('MLSP, Storm Motion, SRH.jpg', bbox_inches="tight")
plt.show()